In [12]:
import glob
import os

In [5]:
files = glob.glob('./data/enframed/**/*.png', recursive=True)
len(files)

1539

In [33]:
import cv2
import traceback
import numpy as np
import math

In [44]:
class ExtractorCR(object):
    """
    Extrator dos blocos de questões do cartão resposta
    """
    def __init__(self, img, templates, num_blocks, expected_rows):
        self.img_width, self.img_height = img.shape[::-1]
        self.width, self.height = templates[0].shape[::-1]
        self.img = img
        self.templates = templates
        self.num_blocks = num_blocks
        self.expected_rows = expected_rows
        self.points_left = []
        self.points_right = []
        self.__threshold = 0.84

    def is_valid(self):
        """
        Verifica se é possível extrair questões da imagem
        """
        self.points_left, self.points_right = self.__get_points()
        return len(self.points_left) > 0 and len(self.points_left) == len(self.points_right)

    def extract(self, output_dir):
        """
        Extrai as questoes e salva no 'output_dir' no formato 'q{num}.png'
        """
        if not self.is_valid():
            #print('Invalid')
            raise Exception('File is not valid for extract')
        segments = self.__get_segments()
        self.__process(segments, output_dir)

    def __process(self, segments, output_dir):
        row = 0
        for p_l, p_r in zip(self.points_left, self.points_right):
            row += 1
            for col in range(self.num_blocks):
                m_l = np.sum(segments[0:col]) if self.num_blocks > 1 else 0
                m_r = np.sum(segments[col:(self.num_blocks - 1)]) if self.num_blocks > 1 else 0
                cp_img = self.img[int(p_l[1]):int(p_r[1]+self.height), int(p_l[0]+self.width+m_l):int(p_r[0]-m_r)]
                cp_img = cv2.resize(cp_img, (370, 50))
                q_num = row + (len(self.points_left) * col)
                filename = output_dir + '/q' + str(q_num) + '.png'
                self.__check_dir(output_dir)
                cv2.imwrite(filename, cp_img)

    def __check_dir(self, folder):
        d_path = os.path.dirname(folder + '/')
        if not os.path.exists(d_path):
            os.makedirs(d_path)

    def __get_segments(self):
        pos_x = self.points_left[0][0] + self.width
        pos_y = self.points_left[0][1]
        p_temp = (pos_x, pos_y)
        length = self.__distance(p_temp, self.points_right[0])
        segment = length // self.num_blocks
        segments = [segment] * self.num_blocks
        pos = 0
        while np.sum(segments) < length:
            segments[pos] += 1
            pos += 1
            if pos == self.num_blocks:
                pos = 0
        return segments

    def __get_points(self):
        minor_adjust = 0.0
        v_adjust = 0.01
        tries = 0
        maximum_tries = 5 #int(100 - ((self.__threshold * 100)/(v_adjust * 100)))
        d_min = 10
        p_l, p_r = [], []
        while True:
            if len(p_l) != self.expected_rows:
                p_l = self.__get_points_left(minor_adjust, d_min)
            if self.expected_rows == len(p_l):
                minor_adjust = 0
                tries = 0
                break
            if self.expected_rows < len(p_l):
                minor_adjust += v_adjust
            else:
                minor_adjust -= v_adjust
            tries += 1
            if tries >= maximum_tries:
                msg = 'Function cannot determine a threshold valid for expected number of rows: '
                raise Exception(msg + str(self.expected_rows))
        while True:
            if len(p_r) != self.expected_rows:
                p_r = self.__get_points_right(p_l, minor_adjust, d_min)
            if self.expected_rows == len(p_r):
                break
            elif self.expected_rows < len(p_r):
                minor_adjust += v_adjust
            else:
                minor_adjust -= v_adjust
            tries += 1
            if tries >= maximum_tries:
                msg = 'Function cannot determine a threshold valid for expected number of rows: '
                raise Exception(msg + str(self.expected_rows))
        #print(p_l, p_r)
        return p_l, p_r

    def __get_points_left(self, minor_adjust, d_min):
        res_points = []
        for template in self.templates:
            points = self.__extract_points(template, minor_adjust, self.img_width // 2)
            #print('points left: ', len(points))
            result = []
            ref_l = None
            l_list_x, l_list_y = [], []
            for point in points:
                valid = True
                for v_point in result:
                    if self.__distance(v_point, point) < d_min:
                        valid = False
                if not valid or point[0] > self.__get_10p_width():
                    continue
                if ref_l is None:
                    ref_l = point
                if math.fabs(ref_l[1] - point[1]) < 5:
                    l_list_x.append(point[0])
                    l_list_y.append(point[1])
                else:
                    p_x = int(np.median(l_list_x))
                    p_y = int(np.median(l_list_y))
                    result.append((p_x, p_y))
                    l_list_x, l_list_y = [], []
                    ref_l = None
            if ref_l is not None:
                p_x = np.mean(l_list_x)
                p_y = np.mean(l_list_y)
                result.append((p_x, p_y))
            if len(res_points) < len(result) or self.expected_rows == len(result):
                res_points = result
            if self.expected_rows == len(res_points):
                break
        #print('result left', len(res_points))
        return res_points

    def __get_points_right(self, p_left, minor_adjust, d_min):
        for template in self.templates:
            points = self.__extract_points(template, minor_adjust, 0)
            #print('points right: ', len(points))
            index = 0
            better_r = None
            result = []
            for point in points:
                valid = True
                for v_point in result:
                    if self.__distance(v_point, point) < d_min:
                        valid = False
                if not valid or point[0] < self.img_width - self.__get_10p_width():
                    continue
                if better_r is None:
                    better_r = point
                if self.__distance(p_left[index], better_r) >= self.__distance(p_left[index], point):
                    better_r = point
                if math.fabs(better_r[1] - point[1]) >= self.width:
                    result.append(better_r)
                    better_r = None
                    index += 1
            if better_r is not None:
                result.append(better_r)
            #print(len(result))
            if self.expected_rows == len(result):
                break
        return result

    def __extract_points(self, template, minor_adjust, p_x):
        cropped = self.img.copy()
        w_cropped = p_x+self.img_width // 2
        cv2.rectangle(cropped, (p_x, 0), (w_cropped, self.img_height), (255, 255, 255), -1)
        res = cv2.matchTemplate(cropped, template, cv2.TM_CCOEFF_NORMED)
        #print('threshold', (self.__threshold + minor_adjust))
        locs = np.where(res >= (self.__threshold + minor_adjust))
        return list(zip(*locs[::-1]))

    def __get_10p_width(self):
        return self.img_width * (0.13)

    def __distance(self, point_a, point_b):
        d_x = point_a[0] - point_b[0]
        d_y = point_a[1] - point_b[1]
        return math.hypot(d_x, d_y)

In [45]:
anchor_templates = []
for anchor in glob.glob('./data/question-templates/**/*.png', recursive=True):
    anchor_templates.append(cv2.imread(anchor, cv2.IMREAD_GRAYSCALE))

for file in files:
    ck_dir = './data/raw-cropped/{0}'.format(os.path.split(os.path.splitext(file)[0])[1])
    if os.path.isdir(ck_dir):
        continue    
    print('Processando:', file)
    try:
        img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)        
        img_height, img_width = img.shape        
        cv2.rectangle(img, (0, 0), (img_width, (((img_height * 2) // 3) - 300)), (255, 255, 255), -1)
        extractor = ExtractorCR(img, anchor_templates, 6, 15)        
        extractor.extract(ck_dir)
    except Exception as e:
        #errors_folder = './errors'
        #check_dir(errors_folder)
        #cv2.imwrite(errors_folder + '/' + filename, img)
        print(traceback.format_exc())

Processando: ./data/enframed/template-card-531_0.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
  File "<ipython-input-44-272f5b122470>", line 88, in __get_points
    raise Exception(msg + str(self.expected_rows))
Exception: Function cannot determine a threshold valid for expected number of rows: 15

Processando: ./data/enframed/template-card-657_0.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
  

Processando: ./data/enframed/template-card-789_0.png
Processando: ./data/enframed/template-card-701_0.png
Processando: ./data/enframed/template-card-794_0.png
Processando: ./data/enframed/template-card-1229_50.png
Processando: ./data/enframed/template-card-776_0.png
Processando: ./data/enframed/template-card-500_0.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
  File "<ipython-input-44-272f5b122470>", line 88, in __get_points
    raise Exception(msg + str(self.expected_rows))
Exception: Function cannot determine a threshold valid for expected number of rows: 15

Processando: ./data/enframed/template-card-1229_35.png
Processando: ./data/enframed/template-card-325_0.png
Traceback (most recent 

Processando: ./data/enframed/template-card-1229_31.png
Processando: ./data/enframed/template-card-524_0.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
  File "<ipython-input-44-272f5b122470>", line 88, in __get_points
    raise Exception(msg + str(self.expected_rows))
Exception: Function cannot determine a threshold valid for expected number of rows: 15

Processando: ./data/enframed/template-card-992_58.png
Processando: ./data/enframed/template-card-1245_0.png
Processando: ./data/enframed/template-card-1022_0.png
Processando: ./data/enframed/template-card-182_14.png
Processando: ./data/enframed/template-card-777_0.png
Processando: ./data/enframed/template-card-66_0.png
Processando: ./data/en

Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
  File "<ipython-input-44-272f5b122470>", line 88, in __get_points
    raise Exception(msg + str(self.expected_rows))
Exception: Function cannot determine a threshold valid for expected number of rows: 15

Processando: ./data/enframed/template-card-579_0.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
  File "<ipython-input-44-272f5b122470>", line 88, in _

Processando: ./data/enframed/template-card-1228_20.png
Processando: ./data/enframed/template-card-1061_0.png
Processando: ./data/enframed/template-card-917_0.png
Processando: ./data/enframed/template-card-109_0.png
Processando: ./data/enframed/template-card-185_6.png
Processando: ./data/enframed/template-card-199_0.png
Processando: ./data/enframed/template-card-1229_45.png
Processando: ./data/enframed/template-card-678_0.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
  File "<ipython-input-44-272f5b122470>", line 88, in __get_points
    raise Exception(msg + str(self.expected_rows))
Exception: Function cannot determine a threshold valid for expected number of rows: 15

Processando: ./data/en

...

Processando: ./data/enframed/template-card-230_0.png
Processando: ./data/enframed/template-card-1005_0.png
Processando: ./data/enframed/template-card-393_0.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
  File "<ipython-input-44-272f5b122470>", line 88, in __get_points
    raise Exception(msg + str(self.expected_rows))
Exception: Function cannot determine a threshold valid for expected number of rows: 15

Processando: ./data/enframed/template-card-992_15.png
Processando: ./data/enframed/template-card-367_0.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in ex

Processando: ./data/enframed/template-card-418_0.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
  File "<ipython-input-44-272f5b122470>", line 88, in __get_points
    raise Exception(msg + str(self.expected_rows))
Exception: Function cannot determine a threshold valid for expected number of rows: 15

Processando: ./data/enframed/template-card-422_0.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
  

Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
  File "<ipython-input-44-272f5b122470>", line 88, in __get_points
    raise Exception(msg + str(self.expected_rows))
Exception: Function cannot determine a threshold valid for expected number of rows: 15

Processando: ./data/enframed/template-card-344_0.png
Processando: ./data/enframed/template-card-368_0.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
  

Processando: ./data/enframed/template-card-1097_0.png
Processando: ./data/enframed/template-card-1229_18.png
Processando: ./data/enframed/template-card-837_0.png
Processando: ./data/enframed/template-card-348_26.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
  File "<ipython-input-44-272f5b122470>", line 88, in __get_points
    raise Exception(msg + str(self.expected_rows))
Exception: Function cannot determine a threshold valid for expected number of rows: 15

Processando: ./data/enframed/template-card-364_0.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in 

Processando: ./data/enframed/template-card-521_0.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
  File "<ipython-input-44-272f5b122470>", line 88, in __get_points
    raise Exception(msg + str(self.expected_rows))
Exception: Function cannot determine a threshold valid for expected number of rows: 15

Processando: ./data/enframed/template-card-348_41.png
Traceback (most recent call last):
  File "<ipython-input-45-50f2650c751c>", line 15, in <module>
    extractor.extract(ck_dir)
  File "<ipython-input-44-272f5b122470>", line 27, in extract
    if not self.is_valid():
  File "<ipython-input-44-272f5b122470>", line 20, in is_valid
    self.points_left, self.points_right = self.__get_points()
 